In [22]:
import pandas as pd
import os
from datetime import datetime
import warnings
from openpyxl import load_workbook
# 抑制特定类型的警告
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=UserWarning)

# 获取用户输入的日期
date_input = input("请输入日期（格式如20240107）：")
date = datetime.strptime(date_input, '%Y%m%d')
# 定义下载目录
download_dir = r'C:\Users\05294\Downloads'
# 打开文件A
#C:\Users\05294\Downloads\外贸信托-盈享致远-2024-01-07.xlsx
file_a_path = f"{download_dir}委外资产明细导入模板-盈享致远1-3号-{date}.xlsx"

# 加载原Excel文件
wb = load_workbook(file_a_path)

# 默认工作表名称，根据实际情况修改
sheet_name = wb.sheetnames[0]  # 假设数据在第一个工作表

# 获取工作表对象
ws = wb[sheet_name]

# 读取所有行和列的数据
for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
    for cell in row:
        value = cell.value
        if isinstance(value, str) and '科目名称' in value:  # 找到对应的列
            col_index = cell.column  # 获取列索引

file_a = pd.read_excel(file_a_path)

# 删除资产代码列中字符串开头不是NHG和ZHG开头的行
file_a = file_a[file_a['资产代码'].str.startswith('NHG') | file_a['资产代码'].str.startswith('ZHG')]





# 遍历所属资管计划名称列去打开对应的excel文件
plan_files = {}
for index, row in file_a.iterrows():
    plan_name = row['所属资管计划名称']
    date_str = date.strftime('%Y%m%d')  # 将日期格式化为20240107格式
    plan_file_name = f"{plan_name}-{date_str}.xlsx"
    plan_file_path = os.path.join(download_dir, plan_file_name)
    if os.path.exists(plan_file_path):
        if plan_file_path not in plan_files:
            plan_files[plan_file_path] = pd.read_excel(plan_file_path,header=3)

        # 获取对应的计划文件
        plan_file = plan_files[plan_file_path]

        # 检查'科目名称'列是否存在于计划文件中
        if '科目名称' not in plan_file.columns:
            print(f"文件 {plan_file_path} 中不存在'科目名称'列")
            continue
        # 逆回购
        if row['资产名称'] == '逆回购':
            subject_name = '买入返售金融资产'
            cost_row = plan_file[plan_file['科目名称'] == subject_name]
            if not cost_row.empty:
                file_a.at[index, '全价市值（元）'] = cost_row['成本'].values[0]
                print(f"文件 {plan_file_path} 中全价市值为{cost_row['成本'].values[0]}")
        # 正回购
        if row['资产名称'] == '正回购':
            subject_name = '卖出回购证券款'
            cost_row = plan_file[plan_file['科目名称'] == subject_name]
            if not cost_row.empty:
                cost_value = cost_row['成本'].values[0]
                total_forward = file_a[(file_a['资产名称'] == '正回购') & (file_a['所属资管计划名称'] == plan_name)]['全价市值（元）'].sum()
                #需要加上所属资管计划名称外贸信托-盈享致远1号集合资金信托计划


                if total_forward != cost_value:
                    for forward_index, forward_row in file_a[(file_a['资产名称'] == '正回购') & (file_a['所属资管计划名称'] == plan_name)].iterrows():
                        if '131809' in forward_row['资产代码']:
                            subject_name_deep = '深交所卖出回购证券'
                            cost_row_deep = plan_file[plan_file['科目名称'] == subject_name_deep]
                            if not cost_row_deep.empty:
                                file_a.at[forward_index, '全价市值（元）'] = round(cost_row_deep['成本'].values[0], 4)
                                print(f"文件 {plan_file_path} 中正回购深交所卖出回购证券全价市值为{cost_row_deep['成本'].values[0]}")
                        elif '204004' in forward_row['资产代码']:
                            subject_name_sh = '上交所卖出回购证券'
                            cost_row_sh = plan_file[plan_file['科目名称'] == subject_name_sh]
                            if not cost_row_sh.empty:
                                file_a.at[forward_index, '全价市值（元）'] = round(cost_row_sh['成本'].values[0], 4)
                                print(f"文件 {plan_file_path} 中正回购上交所卖出回购证券全价市值为{cost_row_sh['成本'].values[0]}")

# 保存修改后的文件A
# 将DataFrame数据写入工作表
for index, row in file_a.iterrows():
    for col_index, value in enumerate(row, start=1):  # 从1开始，因为openpyxl的列索引从1开始
        cell = ws.cell(row=index + 2, column=col_index)  # +2是因为第一行是列名，跳过第一行和第二行
        cell.value = value

# 保存工作簿
wb.save(file_a_path)

文件 C:\Users\05294\Downloads\外贸信托-盈享致远1号集合资金信托计划-20240107.xlsx 中全价市值为300059547.96
文件 C:\Users\05294\Downloads\外贸信托-盈享致远1号集合资金信托计划-20240107.xlsx 中正回购上交所卖出回购证券全价市值为300058575.24
文件 C:\Users\05294\Downloads\外贸信托-盈享致远2号集合资金信托计划-20240107.xlsx 中正回购上交所卖出回购证券全价市值为250048149.21
文件 C:\Users\05294\Downloads\外贸信托-盈享致远3号集合资金信托计划-20240107.xlsx 中正回购深交所卖出回购证券全价市值为138620351.32
文件 C:\Users\05294\Downloads\外贸信托-盈享致远3号集合资金信托计划-20240107.xlsx 中正回购上交所卖出回购证券全价市值为1600312209.4


D:\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
D:\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
D:\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
